In [180]:
!poetry install

Installing dependencies from lock file

No dependencies to install or update

Installing the current project: mlforscience (0.1.0)
If you do not want to install the current project use --no-root.
If you want to use Poetry only for dependency management but not for packaging, you can disable package mode by setting package-mode = false in your pyproject.toml file.
In a future version of Poetry this warning will become an error!


In [181]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [182]:
path = os.path.join('..', 'hotel_reservations.csv')
df = pd.read_csv(path)

.info и прочие описанные в первом ноутбуке данные выводить в данном ноутбуке не буду

In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

# Предобработка данных

In [184]:
# Выбранные переменные
selected_columns = [
    'no_of_children',
    'no_of_weekend_nights',
    'no_of_week_nights',
    'arrival_month',
    'market_segment_type',
    'repeated_guest',
    'required_car_parking_space',
    'booking_status' # Целевая переменная
]

In [185]:
# Фильтруем данные по выбранным столбцам
df = df[selected_columns]
df = df.dropna()

In [186]:
# Кодирование бинарных переменных
df['repeated_guest'] = df['repeated_guest'].astype(int)
df['required_car_parking_space'] = df['required_car_parking_space'].astype(int)

In [187]:
# Кодирование целевой переменной booking_status
# Преобразование категориальных значений в 0 и 1
# Проверка и обработка booking_status
expected_categories = {'Canceled', 'Not_Canceled'}

df['booking_status'] = df['booking_status'].map({'Canceled': 0, 'Not_Canceled': 1})

In [188]:
# Кодирование бинарных переменных
df['repeated_guest'] = df['repeated_guest'].astype(int)
df['required_car_parking_space'] = df['required_car_parking_space'].astype(int)

In [189]:
# Кодирование многоклассовой переменной market_segment_type
ohe = OneHotEncoder(sparse_output=False, drop='first') 
market_segment_encoded = ohe.fit_transform(df[['market_segment_type']])

In [190]:
market_segment_df = pd.DataFrame(
    market_segment_encoded,
    columns=[f"market_segment_{cat}" for cat in ohe.categories_[0][1:]],
    index=df.index  # Сохраняем индексы для корректного объединения
)

In [191]:
# Обновляем df: удаляем старую переменную и добавляем закодированные столбцы
df = pd.concat([df.drop('market_segment_type', axis=1), market_segment_df], axis=1)

In [192]:
# Проверка результата
print("\nОбработанный датафрейм:\n", df.head())


Обработанный датафрейм:
    no_of_children  no_of_weekend_nights  no_of_week_nights  arrival_month  \
0               0                     1                  2             10   
1               0                     2                  3             11   
2               0                     2                  1              2   
3               0                     0                  2              5   
4               0                     1                  1              4   

   repeated_guest  required_car_parking_space  booking_status  \
0               0                           0               1   
1               0                           0               1   
2               0                           0               0   
3               0                           0               0   
4               0                           0               0   

   market_segment_Complementary  market_segment_Corporate  \
0                           0.0                       0.0  

# Разбиение данных

In [193]:
# Целевая переменная и признаки
X = df.drop(columns=['booking_status'])  # Признаки
y = df['booking_status']  # Целевая переменная

In [194]:
# Разделение данных: 80% на тренировочный набор, 20% на тестовый
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,  # Размер тестового набора
    random_state=42,  # Фиксируем seed для воспроизводимости
    stratify=y  # Сохраняем пропорции классов
)

In [195]:
# Проверка размеров полученных наборов
print(f"Размер X_train: {X_train.shape}")
print(f"Размер X_test: {X_test.shape}")
print(f"Размер y_train: {y_train.shape}")
print(f"Размер y_test: {y_test.shape}")

Размер X_train: (29020, 10)
Размер X_test: (7255, 10)
Размер y_train: (29020,)
Размер y_test: (7255,)


# Оцениваем качество константного предсказания

Будем использовать точность, т.к. мы занимаемся задачей классификации (отмена/не отмена)

In [196]:
# Определяем наиболее частотный класс в тренировочных данных
most_frequent_class = y_train.mode()[0]


In [197]:
# Создаем предсказания для тренировочных и тестовых данных
y_train_constant = [most_frequent_class] * len(y_train)
y_test_constant = [most_frequent_class] * len(y_test)

In [198]:
# Оцениваем точность константного предсказания
train_accuracy = accuracy_score(y_train, y_train_constant)
test_accuracy = accuracy_score(y_test, y_test_constant)

In [199]:
# Вывод результатов
print(f"Наиболее частотный класс: {most_frequent_class}")
print(f"Точность на тренировочных данных (константное предсказание): {train_accuracy:.3f}")
print(f"Точность на тестовых данных (константное предсказание): {test_accuracy:.3f}")

Наиболее частотный класс: 1
Точность на тренировочных данных (константное предсказание): 0.672
Точность на тестовых данных (константное предсказание): 0.672


Установлена базовая точность в 67.2%, которую необходимо превзойти

# Построение базовой модели

Для начала попробуем линейную регрессию - она работает быстро и интерпертируемо

In [200]:
# Нормализация данных (требуется для логистической регрессии)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [201]:
# Обучение модели логистической регрессии
logreg = LogisticRegression(random_state=42, max_iter=1000)
logreg.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [202]:
# Предсказания на тренировочных и тестовых данных
y_train_pred = logreg.predict(X_train_scaled)
y_test_pred = logreg.predict(X_test_scaled)

In [203]:
# Оценка качества модели
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

In [204]:
print(f"Точность на тренировочных данных: {train_accuracy:.3f}")
print(f"Точность на тестовых данных: {test_accuracy:.3f}")

Точность на тренировочных данных: 0.676
Точность на тестовых данных: 0.674


In [205]:
# Вывод полного отчета
print("\nОтчет классификации (тестовые данные):")
print(classification_report(y_test, y_test_pred))



Отчет классификации (тестовые данные):
              precision    recall  f1-score   support

           0       0.63      0.01      0.03      2377
           1       0.67      1.00      0.80      4878

    accuracy                           0.67      7255
   macro avg       0.65      0.50      0.42      7255
weighted avg       0.66      0.67      0.55      7255



## Интерпретация отчета классификации
### Качество модели на тестовых данных:

- Точность (accuracy): 67%, что соответствует baseline, но это не лучший показатель, особенно учитывая дисбаланс классов.
- F1-Score для класса 0 (Canceled): Очень низкий (0.03), что указывает на то, что модель практически не предсказывает отмены бронирования.
- F1-Score для класса 1 (Confirmed): Высокий (0.80), так как модель почти всегда предсказывает класс 1.

### Дисбаланс классов:

- Recall для класса 0 (Canceled): 1%, что означает, что модель практически не способна идентифицировать отмены.
- Recall для класса 1 (Confirmed): 100%, что связано с тем, что модель склонна предсказывать наиболее частотный класс.
Это подтверждает, что модель страдает от дисбаланса классов и не оптимальна для задачи.

### Метрики macro avg и weighted avg:

- Macro avg: Средние метрики для классов, взятые без учета их частоты. Показывает, что производительность на обоих классах далека от идеала.
- Weighted avg: Средние метрики, взвешенные по числу объектов в каждом классе. Указывает на перекос в сторону частого класса (класса 1).

## Теперь попробуем DecisionTreeClassifier

In [206]:
# Инициализация и обучение модели
tree_clf = DecisionTreeClassifier(random_state=42, class_weight='balanced')  # Балансировка классов
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight='balanced', random_state=42)

In [207]:
# Предсказания на тренировочных и тестовых данных
y_train_pred = tree_clf.predict(X_train)
y_test_pred = tree_clf.predict(X_test)

In [208]:
# Оценка качества
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

In [209]:
print(f"Точность на тренировочных данных: {train_accuracy:.3f}")
print(f"Точность на тестовых данных: {test_accuracy:.3f}")

Точность на тренировочных данных: 0.640
Точность на тестовых данных: 0.612


In [210]:
# Отчет классификации
print("\nОтчет классификации (тестовые данные):")
print(classification_report(y_test, y_test_pred))


Отчет классификации (тестовые данные):
              precision    recall  f1-score   support

           0       0.45      0.77      0.56      2377
           1       0.82      0.54      0.65      4878

    accuracy                           0.61      7255
   macro avg       0.64      0.65      0.61      7255
weighted avg       0.70      0.61      0.62      7255



## Интерпретация результатов дерева решений
### Качество на тестовых данных:

- Точность (accuracy): 61%, что ниже baseline (67%).

### F1-Score:
- Для класса 0 (Canceled): 0.56, что существенно выше, чем у логистической регрессии (где было 0.03).
- Для класса 1 (Confirmed): 0.65, что ниже, чем у логистической регрессии.

### Дисбаланс в precision и recall:

Для класса 0 (Canceled):
- Recall = 77%: Дерево стало хорошо идентифицировать отмены бронирования.
- Precision = 45%: Модель делает больше ошибок, классифицируя бронирования как 0.

Для класса 1 (Confirmed):
- Recall = 54%: Пропускает часть бронирований, которые должны быть классифицированы как 1.
- Precision = 82%: Точные предсказания для подтвержденных бронирований.

### Средние метрики (macro avg и weighted avg):

- Macro avg: Средние значения метрик для обоих классов, показывают сбалансированное качество. F1-score (0.61) улучшился по сравнению с логистической регрессией.
- Weighted avg: Учитывает дисбаланс классов, отражая вклад каждого класса. F1-score (0.62) ниже baseline, но больше равномерно учитывает оба класса.

# Попробуем RandomForest

In [211]:
# Инициализация и обучение модели
forest_clf = RandomForestClassifier(
    random_state=42, 
    class_weight='balanced',  # Балансировка классов
    n_estimators=100,  # Количество деревьев
    max_depth=10,  # Ограничение глубины деревьев
    min_samples_leaf=5  # Минимальное количество объектов в листе
)
forest_clf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=5, random_state=42)

In [212]:
# Предсказания на тренировочных и тестовых данных
y_train_pred = forest_clf.predict(X_train)
y_test_pred = forest_clf.predict(X_test)

In [213]:
# Оценка качества
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

In [214]:
print(f"Точность на тренировочных данных: {train_accuracy:.3f}")
print(f"Точность на тестовых данных: {test_accuracy:.3f}")

Точность на тренировочных данных: 0.572
Точность на тестовых данных: 0.564


In [215]:
# Отчет классификации
print("\nОтчет классификации (тестовые данные):")
print(classification_report(y_test, y_test_pred))


Отчет классификации (тестовые данные):
              precision    recall  f1-score   support

           0       0.42      0.84      0.56      2377
           1       0.85      0.43      0.57      4878

    accuracy                           0.56      7255
   macro avg       0.63      0.64      0.56      7255
weighted avg       0.71      0.56      0.57      7255



## Интерпретация результатов RandomForestClassifier
### Качество модели:

- Точность (accuracy): 56%, что ниже baseline (67%).

### F1-score:
- Для класса 0 (Canceled): 0.56, значительно выше, чем у логистической регрессии, но на уровне дерева решений.
- Для класса 1 (Confirmed): 0.57, также улучшение по сравнению с логистической регрессией, но хуже по сравнению с некоторыми настройками дерева решений.

### Дисбаланс precision и recall:

Для класса 0 (Canceled):

- Recall = 84%: Модель хорошо идентифицирует отмены бронирований.
- Precision = 42%: Значительное количество ложных срабатываний.

Для класса 1 (Confirmed):

- Recall = 43%: Пропускает больше половины бронирований, которые должны быть классифицированы как 1.
- Precision = 85%: Очень точные предсказания для подтвержденных бронирований, но модель пропускает многие.

### Средние метрики (macro avg и weighted avg):

Macro avg:

- F1-score = 0.56, сбалансированное качество на обоих классах.

Weighted avg:

- F1-score = 0.57, с учетом веса более частого класса.

# Общий вывод: Почему модели показали результат хуже бейзлайна?

## Основные причины:

### 1. Дисбаланс классов
- Бейзлайн (константное предсказание наиболее частого класса) достиг точности **67%** за счет того, что весь тестовый набор классифицировался в класс `1` (Not_Canceled), который встречается значительно чаще.
- Модели, стремясь предсказывать оба класса, испытывают трудности из-за дисбаланса: предсказания класса `0` (Canceled) снижают общую точность.

### 2. Метрика `accuracy` вводит в заблуждение
- Концентрация на `accuracy` как основной метрики не отражает реальной эффективности моделей в задаче, где важны оба класса.
- `F1-score (macro)` показал, что модели лучше балансируют классы по сравнению с бейзлайном, но это не отразилось на общей точности.

### 3. Переобучение и недообучение
- Дерево решений и случайный лес могли либо переобучиться на менее частых признаках класса `0`, либо не учли их в полной мере из-за недостаточной глубины/настроек.

### 4. Масштаб признаков
- Логистическая регрессия, будучи линейной моделью, чувствительна к масштабированию данных. Хотя мы использовали нормализацию, линейные модели часто имеют ограниченные возможности для сложных задач, где присутствуют нелинейные зависимости.

### 5. Недостаточная настройка гиперпараметров
- Параметры, такие как глубина дерева, количество деревьев в случайном лесу или веса классов, могли быть выбраны не оптимально.

---

## Что можно с этим сделать?

### 1. Балансировка классов
- Использовать стратегии:
  - **Class weights**: Уже применялись, но можно усилить вес менее частого класса.
  - **Oversampling**: Добавить искусственные данные для класса `0` с помощью методов вроде SMOTE.
  - **Undersampling**: Уменьшить количество объектов класса `1`.

### 2. Выбор метрики
- Сконцентрироваться на `F1-score (macro)` или `F1-score (weighted)` вместо `accuracy`.
- Использовать `precision-recall curve` или `ROC-AUC` для оценки моделей в условиях дисбаланса.

### 3. Гиперпараметры моделей
- Подбор глубины дерева, минимального количества объектов в листьях и других параметров для деревьев решений и случайного леса.
- Увеличение количества деревьев (`n_estimators`) в случайном лесе.

### 4. Более сложные модели
- Применить алгоритмы, такие как **XGBoost** или **LightGBM**, которые лучше справляются с дисбалансом и учитывают взаимодействия между признаками.

### 5. Анализ данных
- Пересмотреть и при необходимости расширить выбор признаков. Возможно, текущие признаки недостаточно хорошо объясняют поведение класса `0`.

### 6. Сегментация данных
- Разделить данные на сегменты (например, по месяцам, категориям бронирования) и обучать модели отдельно на каждом сегменте.

---

## Итог:
Основная проблема — **дисбаланс классов** и его влияние на метрику точности. 

### Основные направления улучшения:
1. **Оптимизация метрики**: использовать более подходящие метрики вместо точности.
2. **Балансировка классов**: применить oversampling или undersampling.
3. **Использование сложных моделей**: XGBoost, LightGBM.
4. **Подбор гиперпараметров**: улучшить текущие настройки моделей.
